# L.Point 구매자 집계

In [2]:
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

database = os.getenv('CON_STR')

engine = create_engine(database)


In [25]:
# 1. 첨부파일의 제품들의 출시 첫 달부터 purchaser 수
sql = '''
select engname, 
	b.FLAVORSEG_type3,
	t.YYYYMM, 
	count(distinct t.id) Purchasers
from cx.v_user_3month_list t
	join cx.fct_K7_Monthly a on a.id = t.id and a.YYYYMM = t.YYYYMM 
	join cx.product_master b on a.product_code = b.PROD_ID  and CIGADEVICE ='CIGARETTES' and CIGATYPE = 'CC'
where t.YYYYMM >= '202201'	
group by  engname, b.FLAVORSEG_type3, t.YYYYMM
'''

df = pd.read_sql_query(sql, engine)

pivot_df = df.pivot_table(index=['engname', 'FLAVORSEG_type3'],
							columns='YYYYMM',
							values='Purchasers').reset_index()

pivot_df

YYYYMM,engname,FLAVORSEG_type3,202201,202202,202203,202204,202205,202206,202207,202208,...,202310,202311,202312,202401,202402,202403,202404,202405,202406,202407
0,88 ICE GOLD,Fresh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,88 RETURNS,Regular,365.0,340.0,328.0,370.0,372.0,352.0,379.0,380.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BOHEM CIGAR CARIBE,New Taste,1140.0,1024.0,1053.0,1076.0,1150.0,1163.0,1229.0,1219.0,...,1567.0,1553.0,1455.0,1450.0,1400.0,1534.0,1637.0,1607.0,1694.0,1760.0
3,BOHEM CIGAR CUBANA DOUBLE,New Taste,892.0,833.0,926.0,1053.0,1102.0,1152.0,1289.0,1248.0,...,1410.0,1308.0,1301.0,1211.0,1189.0,1320.0,1429.0,1415.0,1475.0,1520.0
4,BOHEM CIGAR ICE FIT,New Taste,NaN,NaN,1960.0,1480.0,1195.0,1123.0,1102.0,1058.0,...,1260.0,1263.0,1230.0,1226.0,1185.0,1255.0,1369.0,1509.0,1449.0,1526.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,TIMELESS TIME KS BOX 20,Regular,248.0,256.0,261.0,263.0,258.0,282.0,279.0,258.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,VS S. BLUE,Regular,156.0,154.0,167.0,180.0,191.0,180.0,206.0,207.0,...,201.0,187.0,192.0,202.0,207.0,198.0,211.0,208.0,205.0,213.0
225,VS S. GOLD,Regular,1084.0,939.0,1082.0,1195.0,1153.0,1221.0,1182.0,1209.0,...,1372.0,1272.0,1265.0,1280.0,1172.0,1258.0,1278.0,1345.0,1326.0,1340.0
226,VS S. ONE,Regular,489.0,477.0,502.0,511.0,569.0,543.0,548.0,546.0,...,563.0,555.0,554.0,537.0,513.0,518.0,544.0,557.0,561.0,585.0


In [28]:
pivot_df.to_clipboard(index=False)

In [30]:
#  2. 첨부파일의 제품들의 출시 첫 달부터 재구매 purchaser 수 (해당 제품 2팩이상 구매자)
sql2 = '''
with temp as ( 
	-- 해당 제품 2팩이상 구매자
	select engname,
		b.FLAVORSEG_type3,
		t.YYYYMM, 
		t.id,
		sum(a.Pack_qty) pack
	from cx.v_user_3month_list t
		join cx.fct_K7_Monthly a on a.id = t.id and a.YYYYMM = t.YYYYMM 
		join cx.product_master b on a.product_code = b.PROD_ID  and CIGADEVICE ='CIGARETTES' and CIGATYPE = 'CC'
	where t.YYYYMM >= '202201'
	group by engname, b.FLAVORSEG_type3,t.YYYYMM, t.id
	having sum(a.Pack_qty ) > 1
)
select engname,
	FLAVORSEG_type3,
	YYYYMM,
	count(distinct id) Purchasers
from temp
group by engname, FLAVORSEG_type3, YYYYMM
'''

df2 = pd.read_sql_query(sql2, engine)

pivot_df2 = df2.pivot_table(index=['engname', 'FLAVORSEG_type3'],
							columns='YYYYMM',
							values='Purchasers').reset_index()

pivot_df2

YYYYMM,engname,FLAVORSEG_type3,202201,202202,202203,202204,202205,202206,202207,202208,...,202310,202311,202312,202401,202402,202403,202404,202405,202406,202407
0,88 ICE GOLD,Fresh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,88 RETURNS,Regular,190.0,167.0,167.0,181.0,205.0,185.0,206.0,218.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BOHEM CIGAR CARIBE,New Taste,606.0,561.0,554.0,596.0,603.0,634.0,658.0,658.0,...,873.0,828.0,809.0,825.0,787.0,866.0,883.0,901.0,913.0,985.0
3,BOHEM CIGAR CUBANA DOUBLE,New Taste,444.0,446.0,488.0,532.0,582.0,594.0,642.0,586.0,...,748.0,704.0,660.0,635.0,648.0,708.0,759.0,780.0,766.0,786.0
4,BOHEM CIGAR ICE FIT,New Taste,NaN,NaN,626.0,682.0,591.0,627.0,586.0,556.0,...,706.0,711.0,703.0,677.0,688.0,716.0,790.0,877.0,832.0,903.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,TIME MID,Regular,218.0,200.0,235.0,237.0,235.0,229.0,260.0,253.0,...,376.0,358.0,345.0,365.0,352.0,324.0,361.0,371.0,383.0,360.0
217,TIMELESS TIME KS BOX 20,Regular,166.0,185.0,170.0,177.0,174.0,179.0,189.0,173.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,VS S. BLUE,Regular,93.0,97.0,108.0,122.0,117.0,122.0,133.0,134.0,...,128.0,125.0,123.0,130.0,131.0,134.0,145.0,141.0,135.0,125.0
219,VS S. GOLD,Regular,718.0,621.0,693.0,801.0,744.0,775.0,767.0,765.0,...,914.0,852.0,842.0,843.0,773.0,849.0,863.0,864.0,859.0,875.0


In [33]:
pivot_df2.to_clipboard(index=False)

In [31]:
#  3. 첨부파일의 제품들의 출시 첫 달부터 Pack 수
sql3 = '''
select engname, 
	b.FLAVORSEG_type3,
	t.YYYYMM, 
	 sum(a.Pack_qty) Pack_Qty
from cx.v_user_3month_list t
	join cx.fct_K7_Monthly a on a.id = t.id and a.YYYYMM = t.YYYYMM 
	join cx.product_master b on a.product_code = b.PROD_ID  and CIGADEVICE ='CIGARETTES' and CIGATYPE = 'CC'
where t.YYYYMM >= '202201'	
group by engname, b.FLAVORSEG_type3, t.YYYYMM
'''

df3 = pd.read_sql_query(sql3, engine)

pivot_df3 = df3.pivot_table(index=['engname', 'FLAVORSEG_type3'],
							columns='YYYYMM',
							values='Pack_Qty').reset_index()

pivot_df3

YYYYMM,engname,FLAVORSEG_type3,202201,202202,202203,202204,202205,202206,202207,202208,...,202310,202311,202312,202401,202402,202403,202404,202405,202406,202407
0,88 ICE GOLD,Fresh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,88 RETURNS,Regular,1092.0,959.0,1060.0,1063.0,1283.0,1201.0,1212.0,1286.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BOHEM CIGAR CARIBE,New Taste,3597.0,3174.0,3594.0,3676.0,3764.0,3774.0,4030.0,3941.0,...,4952.0,4943.0,4521.0,4719.0,4180.0,4976.0,5208.0,5405.0,5622.0,5979.0
3,BOHEM CIGAR CUBANA DOUBLE,New Taste,2619.0,2351.0,2789.0,3213.0,3443.0,3326.0,3665.0,3452.0,...,4120.0,3991.0,3794.0,3632.0,3571.0,3933.0,4254.0,4463.0,4469.0,4789.0
4,BOHEM CIGAR ICE FIT,New Taste,NaN,NaN,3733.0,3806.0,3554.0,3506.0,3329.0,3170.0,...,4045.0,4070.0,4132.0,4068.0,3681.0,4141.0,4517.0,4981.0,5044.0,5375.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,TIMELESS TIME KS BOX 20,Regular,995.0,972.0,1057.0,1044.0,1079.0,1142.0,1128.0,1012.0,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,VS S. BLUE,Regular,572.0,594.0,679.0,704.0,695.0,688.0,830.0,781.0,...,716.0,640.0,729.0,859.0,758.0,741.0,851.0,789.0,731.0,805.0
225,VS S. GOLD,Regular,4089.0,3426.0,4192.0,4411.0,4532.0,4629.0,4773.0,4586.0,...,5462.0,5058.0,4832.0,4716.0,4249.0,5254.0,5341.0,5458.0,5318.0,5340.0
226,VS S. ONE,Regular,1804.0,1807.0,2050.0,1757.0,1930.0,1920.0,2039.0,2110.0,...,2339.0,2168.0,2181.0,2175.0,2116.0,2286.0,2236.0,2342.0,2289.0,2346.0


In [34]:
pivot_df3.to_clipboard(index=False)

In [32]:
#  4. 첨부파일의 제품들의 출시 첫 달부터 재구매 Pack 수 (해당 제품 2팩이상 구매자)
sql4 = '''
with temp as ( 
	-- 해당 제품 2팩이상 구매자
	select engname,
		b.FLAVORSEG_type3,
		t.YYYYMM, 
		t.id,
		sum(a.Pack_qty) pack
	from cx.v_user_3month_list t
		join cx.fct_K7_Monthly a on a.id = t.id and a.YYYYMM = t.YYYYMM 
		join cx.product_master b on a.product_code = b.PROD_ID  and CIGADEVICE ='CIGARETTES' and CIGATYPE = 'CC'
	where t.YYYYMM >= '202201'
	group by engname, b.FLAVORSEG_type3, t.YYYYMM, t.id
	having sum(a.Pack_qty ) > 1
)
select engname,
	FLAVORSEG_type3,
	YYYYMM,
	 sum(pack) Pack_Qty
from temp
group by engname, FLAVORSEG_type3, YYYYMM
'''

df4 = pd.read_sql_query(sql4, engine)

pivot_df4 = df4.pivot_table(index=['engname','FLAVORSEG_type3'],
							columns='YYYYMM',
							values='Pack_Qty').reset_index()

pivot_df4

YYYYMM,engname,FLAVORSEG_type3,202201,202202,202203,202204,202205,202206,202207,202208,...,202310,202311,202312,202401,202402,202403,202404,202405,202406,202407
0,88 ICE GOLD,Fresh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,88 RETURNS,Regular,917.0,786.0,899.0,874.0,1116.0,1034.0,1039.0,1124.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BOHEM CIGAR CARIBE,New Taste,3063.0,2711.0,3095.0,3196.0,3217.0,3245.0,3459.0,3380.0,...,4258.0,4218.0,3875.0,4094.0,3567.0,4308.0,4454.0,4699.0,4841.0,5204.0
3,BOHEM CIGAR CUBANA DOUBLE,New Taste,2171.0,1964.0,2351.0,2692.0,2923.0,2768.0,3018.0,2790.0,...,3458.0,3387.0,3153.0,3056.0,3030.0,3321.0,3584.0,3828.0,3760.0,4055.0
4,BOHEM CIGAR ICE FIT,New Taste,NaN,NaN,2399.0,3008.0,2950.0,3010.0,2813.0,2668.0,...,3491.0,3518.0,3605.0,3519.0,3184.0,3602.0,3938.0,4349.0,4427.0,4752.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,TIME MID,Regular,1082.0,975.0,1242.0,1416.0,1435.0,1561.0,1574.0,1474.0,...,2262.0,2355.0,2018.0,2180.0,1991.0,1973.0,2004.0,2094.0,2164.0,2247.0
217,TIMELESS TIME KS BOX 20,Regular,913.0,901.0,966.0,958.0,995.0,1039.0,1038.0,927.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,VS S. BLUE,Regular,509.0,537.0,620.0,646.0,621.0,630.0,757.0,708.0,...,643.0,578.0,660.0,787.0,682.0,677.0,785.0,722.0,661.0,717.0
219,VS S. GOLD,Regular,3723.0,3108.0,3803.0,4017.0,4123.0,4183.0,4358.0,4142.0,...,5004.0,4638.0,4409.0,4279.0,3850.0,4845.0,4926.0,4977.0,4851.0,4875.0


In [35]:
pivot_df4.to_clipboard(index=False)